In [9]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [10]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Choose important cases to generate, important scenario is the scenario that users are more likely to encounter it.
- Limit the appearance of scenarios that are hard to happen. 
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and use c"invalid": "below 8 characters, over 30 chaase specification.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""


FILTER_SYSTEM_PROMPT="""
Your task is to read all of this scenarios generation from many sources.
Then you remove all the same copies because there are many duplicate scenarios in meanings inside the input.
Make sure every scenarios in the response is unique.
Return scenario name only.
"""

# Identify what type of that element (button,buttons, icon,scroll, text field, radio buttons, menu, menu for navigation,menu for opening dialog or another menu, menu for filter, sliders, switches, dialog, link, form,rating, filter).


In [11]:
main_flow_prompt="""
Main scenario:
User clicks the "Create new" button in the book list page;
Application displays form to enter book data;
User enters book data and presses "Submit" button;
If any data is entered incorrectly, incorrect data messages are displayed;
If entered data is valid, then record is adding to database;
If error occurs, then error message is displaying;
If new book record is successfully added, then list of books with added records is displaying.
"""

alt_prompt_1="""
Main scenario:
User clicks the "Create new" button in the book list page;
Application displays form to enter book data;
User enters book data and presses "Submit" button;
If any data is entered incorrectly, incorrect data messages are displayed;
If entered data is valid, then record is adding to database;
If error occurs, then error message is displaying;
If new book record is successfully added, then list of books with added records is displaying.

Cancel operation scenario:
User clicks the "Create new" button in the book list page;
Application displays form to enter book data;
User may start entering the data into the form fields;
Before pressing "Submit" button user presses “Cancel” button;
Data doesn’t save in the database, then a list of books records is displayed to the user;
If the user selects the menu item "Books”, "Genres”, "Authors", "Home" or "API", the data will not be saved to the database, and the corresponding form with updated data will be opened.

"""

alt_prompt_2="""
Main scenario:
User clicks the "Create new" button in the book list page;
Application displays form to enter book data;
User enters book data and presses "Submit" button;
If any data is entered incorrectly, incorrect data messages are displayed;
If entered data is valid, then record is adding to database;
If error occurs, then error message is displaying;
If new book record is successfully added, then list of books with added records is displaying.

When creating a new book, the following details are entered:
Title
Author
Genre
Description
Publication date
"""

prompt_all="""
use case name: Add a new book
Main scenario:
User clicks the "Create new" button in the book list page;
Application displays form to enter book data;
User enters book data and presses "Submit" button;
If any data is entered incorrectly, incorrect data messages are displayed;
If entered data is valid, then record is adding to database;
If error occurs, then error message is displaying;
If new book record is successfully added, then list of books with added records is displaying.

Cancel operation scenario:
User clicks the "Create new" button in the book list page;
Application displays form to enter book data;
User may start entering the data into the form fields;
Before pressing "Submit" button user presses “Cancel” button;
Data doesn’t save in the database, then a list of books records is displayed to the user;
If the user selects the menu item "Books”, "Genres”, "Authors", "Home" or "API", the data will not be saved to the database, and the corresponding form with updated data will be opened.

When creating a new book, the following details are entered:
Title
Author
Genre
Description
Publication date
"""

In [12]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

filtercontent = main_gpt_response + "/n" + sub1_gpt_response + "/n" + sub2_gpt_response

# promptFilter = [
#     { "role": "system", "content": FILTER_SYSTEM_PROMPT},
#     { "role": "user", "content": filtercontent}
#   ]
# filter_gpt_response = ask(promptFilter, client, model)
# print(filter_gpt_response)

In [13]:
promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

{'Title': {'condition': {'valid': 'must be entered', 'invalid': 'empty'}, 'type': 'text field'}, 'Author': {'condition': {'valid': 'must be entered', 'invalid': 'empty'}, 'type': 'text field'}, 'Genre': {'condition': {'valid': 'must be entered', 'invalid': 'empty'}, 'type': 'text field'}, 'Description': {'condition': {'valid': 'must be entered', 'invalid': 'empty'}, 'type': 'text area'}, 'Publication date': {'condition': {'valid': 'must be entered', 'invalid': 'empty'}, 'type': 'text field'}}


In [14]:
if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)
  # filtercontent+= "/n" + condition_scenario_response
  # print(filtercontent)

1. Scenario: Add a new book with empty title
2. Scenario: Add a new book with empty author
3. Scenario: Add a new book with empty genre
4. Scenario: Add a new book with empty description
5. Scenario: Add a new book with empty publication date


In [15]:
filtercontent = main_gpt_response + "\n" + sub1_gpt_response + "\n" + sub2_gpt_response + "\n" + condition_scenario_response
print(filtercontent)

1. Successful Book Creation
2. Book Creation with Incorrect Data
3. Book Creation Failure Due to System Error
Based on the provided main and alternative (exception) flow, here are the scenarios that can lead a user from the main flow to change to the alternative or exception flow:

1. **Cancel Operation Before Data Submission**: This scenario starts with the user clicking the "Create new" button and entering data into the form but decides to press the "Cancel" button before submitting the data.

2. **Navigation Away Using Menu Before Submission**: In this scenario, the user begins to enter book data after clicking "Create new" but then navigates away from the page by selecting another menu item like "Books", "Genres", "Authors", "Home", or "API" before pressing the "Submit" button.
Based on the provided main flow and the rules for predicting scenarios that could lead a user from the main flow to an alternative or exception flow, here are the potential scenarios:

1. Incorrect Data Entr

In [16]:
promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print(filter_gpt_response)

- Successful Book Creation
- Book Creation with Incorrect Data
- Book Creation Failure Due to System Error
- Cancel Operation Before Data Submission
- Navigation Away Using Menu Before Submission
- Incorrect Data Entry Scenario
- Database Error Scenario
- Add a new book with empty title
- Add a new book with empty author
- Add a new book with empty genre
- Add a new book with empty description
- Add a new book with empty publication date
